<img src="logo.png">

# Máquinas de Vectores Soporte (Support Vector Machines, ó SVM)

Vamos a ver como utiliar el algoritmo SVM en `scikit-learn`. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [10, 10]

### Cargamos los datos

En este caso vamos a usar un dataset nuevo, el dataset [Iris](https://archive.ics.uci.edu/ml/datasets/iris), que contiene 4 características sobre 3 tipos distintos de variedades de la flor Iris

In [ ]:
from sklearn.datasets import load_iris

datos = load_iris()

In [ ]:
print(datos.DESCR)

In [ ]:
datos.target_names

In [ ]:
datos.target

In [ ]:
iris = pd.DataFrame(datos.data, columns=datos.feature_names)

iris["objetivo"] = datos.target

In [ ]:
iris.head()

In [ ]:
variables_indep = iris.drop("objetivo", axis=1).columns

iris_X = iris[variables_indep]
iris_y = iris["objetivo"]

In [ ]:
from sklearn.model_selection import train_test_split

iris_X_train, iris_X_test, iris_y_train, iris_y_test = train_test_split(iris_X, iris_y, test_size=0.2)

In [ ]:
from sklearn.svm import SVC, SVR

In [ ]:
estimador_svm =  SVC()

In [ ]:
estimador_svm.fit(iris_X_train, iris_y_train)

In [ ]:
estimador_svm.predict(iris_X_test)[:10]

- **C** es el parámetro de Coste (que regula el impacto de las variables de holgura y ayuda a regularizar el modelo.
- **kernel** indica que kernel usar (rbf, radial basis function por defecto). Se puede usar cualquier kernel definido por nosotros, por defecto SVC conoce `rbf`, `poly` (polinomial), `linear` (lineal) o `sigmoid` (sigmoide).
- **class_weight**, nos permite pasar un diccionario de la forma `{clase:peso}` que permite asignar más peso a una clase que a otra. Para problemas con clases no balanceadas, podemos usar el parámetro 'balanced' para que se ajusten los pesos en función del número de casos de cada clase. 
- **decision_function_shape** si usar una estrategia de uno contra uno (ovo) o uno contra todos (one versus rest, ovr) en casos de clasificación multiclase.
- **cache_size** es el tamaño (en megabytes) del caché del modelo (cuantos datos puede guardar en memoria y reutilizarlos sin tener que calcularlos). SVMs son computacionalmente complejos asi que si hay mas memoria disponible mejor incrementarl este valor(por ejemplo, a 1000mb o 2000mb)

El parámetro `support_vectors_` nos devuelve los vectores soporte.

In [ ]:
estimador_svm.support_vectors_

El parámetro `n_support_` nos dice cuantos vectores soporte (es decir, puntos tocando el margen del hiperplano de decisión) existen por clase, esto nos da una medida de cuan faciles son de separar cada clase.

In [ ]:
estimador_svm.n_support_

# Kernels

Vamos a ver el efecto que tienen diferentes kernels en la creación del hiperplano de decisión.

Para verlo en un gráfico de dispersión tomamos solo las dos primeras variables del dataset (longitud y grosor del sépalo).

In [ ]:
X = datos.data[:, :2]
y = datos.target

In [ ]:
!pip install mlxtend

In [ ]:
from mlxtend.plotting import plot_decision_regions

En primer lugar tenemos el kernel lineal

In [ ]:
estimador_svm_lineal = SVC(kernel="linear")
estimador_svm_lineal.fit(X, y)

plot_decision_regions(X, y, clf=estimador_svm_lineal);

El kernel polinomial calcula el producto escalar de dos vectores en un espacio dimensional donde tiene en cuenta las combinaciones polinómicas de los mismos. Esto es, si tenemos dos vectores V1 y V2 de la forma `[x1, x2]` el kernel polinomial va a producir un resultado similar a hacer un producto escalar de V1 y V2 pero transformados como `[x1, x2, x1^2, x1x2, x2^2...]`.

El kernel polinomial tiene el hiperparámetro `degree` que indica el grado de expansión polinómica (esto es, el grado de las combinaciones de las variables que queremos tener en cuenta). Por defecto es 3.

In [ ]:
estimador_svm_polinomial = SVC(kernel="poly")
estimador_svm_polinomial.fit(X, y)

plot_decision_regions(X, y, clf=estimador_svm_polinomial);

Podemos ver como varia la frontera de decisión en función de los grados de expansión. Cuantos más grados más complejo podrá ser el hiperplano. Con grado 1 se convierte en un kernel lineal.

In [ ]:
estimador_svm_polinomial = SVC(kernel="poly", degree=1).fit(X, y)
plot_decision_regions(X, y, clf=estimador_svm_polinomial);

In [ ]:
estimador_svm_polinomial = SVC(kernel="poly", degree=2).fit(X, y)
plot_decision_regions(X, y, clf=estimador_svm_polinomial);

In [ ]:
estimador_svm_polinomial = SVC(kernel="poly", degree=6).fit(X, y)
plot_decision_regions(X, y, clf=estimador_svm_polinomial);

El kernel polinomial tiene también el parámetro que lo regula, alpha en la ecuación de arriba, aunque en la implementación de scikit-learn se llama `gamma` 

In [ ]:
estimador_svm_polinomial = SVC(kernel="poly", degree=3, gamma=0.1).fit(X, y)
plot_decision_regions(X, y, clf=estimador_svm_polinomial);

Vemos que un gamma bajo reduce la complejidad del kernel polinomial convirtiendolo prácticamente en lineal. Si no se especifica se usa `gamma=1/n_variables`, o sea en este caso 1/2

El kernel gausiano (radial basis function, o rbf) hace una transformacion radial (esto es, en funcion de la distancia de los puntos al origen).

In [ ]:
estimador_svm_rbf = SVC(kernel="rbf")
estimador_svm_rbf.fit(X, y)

plot_decision_regions(X, y, clf=estimador_svm_rbf);

Podemos probar como varia la frontera de decisión en función de gamma:

In [ ]:
estimador_svm_rbf = SVC(kernel="rbf", gamma=0.1).fit(X, y)
plot_decision_regions(X, y, clf=estimador_svm_rbf);

In [ ]:
estimador_svm_rbf = SVC(kernel="rbf", gamma=10).fit(X, y)
plot_decision_regions(X, y, clf=estimador_svm_rbf);

In [ ]:
estimador_svm_rbf = SVC(kernel="rbf", gamma=100).fit(X, y)
plot_decision_regions(X, y, clf=estimador_svm_rbf);

A mayores valores de `gamma` mayor es la capacidad del kernel rbf de crear segmentos alrededor de los datos. 
Vemos que para `gamma=100` el modelo está sobreajustando mucho (creando pequeñas burbujas alrededor de cada observación).

# Parámetro de coste C

El parámetro C nos da una medida de como queremos penalizar al modelo cuando clasifica un ejemplo de forma errónea y es la manera de regularizar los modelos SVM. Valores altos de C permiten controlar la complejidad del modelo (evitando el sobreajuste) a coste de no clasificar bien un porcentaje de los ejemplos en los datos de entrenamiento

In [ ]:
from sklearn.model_selection import validation_curve

In [ ]:
rango_c = np.linspace(0.01,50,50)

In [ ]:
estimador_svm

In [ ]:
train_scores, test_scores = validation_curve(estimador_svm, iris_X, iris_y, param_name="C",
                                             param_range=rango_c, cv=10, scoring="f1_weighted")

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.plot(rango_c, train_scores_mean, 'o-', color="r",
             label="Funcionamiento datos_entrenamiento")
plt.plot(rango_c, test_scores_mean, 'o-', color="g",
             label="Funcionamiento Validación Cruzada")
plt.title("Curva Validación: SVM con kernel rbf")
plt.xlabel("Constante de regularización (C)")
plt.ylabel("Puntuación F1")
plt.legend();

Vemos que conforme aumenta C, el modelo sobreajusta más (ya que mejora su funcionamiento en los datos de entrenamiento pero empeora en los de test).

 # Probabilidades

Los modelos SVM no proporcionan probabilidades (por que no hacen inferencia estadística en ese sentido, sino que funcionan de un modo geométrico), por eso por defecto el modelo `SVC` no proporciona el metodo `predict_proba` que hemos visto en otros estimadores (regresión logística por ejemplo).

Sin embargo, la implementación de sklearn permite pasarle el parámetro `probability=True` que calcula de forma adicional las probabilidades usando escalado de Platt (básicamente, entrena una regresión logística en las distancias al hiperplano computadas por el SVM).

Éste método es computacionalmente complejo, y además tiene ciertos fallos teóricos (por ejemplo, puede haber casos en los que se prediga una clase en un problema de clasificación binaria y que su método `predict_proba` produzca una probabilidad menor que 0.5). 

Para aquellos casos que se necesite una forma de puntuar nuevas observaciones pero que dicha puntuacion no tenga que ser una probabilidad es mejor usar directamente el output de la función de decisión con `decision_function`

In [ ]:
estimador_svm =  SVC()

estimador_svm.fit(iris_X_train, iris_y_train)

In [ ]:
estimador_svm.predict(iris_X_test)[:10]

In [ ]:
estimador_svm.decision_function(iris_X_test)[:10]

In [ ]:
estimador_svm.predict_proba(iris_X_test)[:10]

In [ ]:
estimador_svm_prob =  SVC(probability=True)
estimador_svm_prob.fit(iris_X_train, iris_y_train)

In [ ]:
estimador_svm_prob.predict_proba(iris_X_test)[:10]

Vemos que usando `probability=True` se pueden calcular las "probabilidades"